In [14]:
%pwd

'c:\\Users\\nipun\\Desktop\\Medchatbot'

In [15]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os 

In [16]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [17]:
extracted_data=load_pdf_file(data='Data/')

In [18]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [19]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 39711


In [20]:
from langchain.embeddings import HuggingFaceEmbeddings

In [21]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [22]:
embeddings = download_hugging_face_embeddings()

C:\Users\nipun\AppData\Local\Temp\ipykernel_33780\1196424635.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [23]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [24]:
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')

In [26]:
import os
from pinecone import Pinecone, ServerlessSpec

# Your Pinecone API key
PINECONE_API_KEY = "pcsk_6hfbZM_PyDhaPVXZs1H5ddoLnsEWx7iNai3SZR8Mfbgsf59yEKteEmXsiatUEu5q2RPML2"  # Replace with your actual API key

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Define the index name
index_name = "medbot"

# Create the index if it doesn't exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # Adjust based on your embeddings
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

print(f"Index '{index_name}' is ready!")


Index 'medbot' is ready!


In [27]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [29]:
import torch
from sentence_transformers import SentenceTransformer
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings.base import Embeddings

# Ensure PyTorch uses GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the embedding model and move it to GPU
embedding_model = SentenceTransformer("all-MiniLM-L6-v2", device="cuda")

# Custom embedding class for LangChain compatibility
class CustomEmbeddings(Embeddings):
    def embed_documents(self, texts):
        with torch.no_grad():
            return embedding_model.encode(texts, convert_to_tensor=True, device=device).cpu().numpy().tolist()

    def embed_query(self, text):
        return self.embed_documents([text])[0]

# Instantiate the custom embedding class
embedding_instance = CustomEmbeddings()

# Batch processing for Pinecone upsert
batch_size = 1000  # Adjust as needed
for i in range(0, len(text_chunks), batch_size):
    docsearch = PineconeVectorStore.from_documents(
        documents=text_chunks[i:i+batch_size],
        index_name=index_name,
        embedding=embedding_instance  # Use the instance, not a function
    )


In [30]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [31]:
docsearch

In [32]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [33]:
retrieved_docs = retriever.invoke("What is Cancer?")

In [ ]:
retrieved_docs

  Using cached pinecone_plugin_inference-1.1.0-py3-none-any.whl.metadata (2.2 kB)
INFO: pip is looking at multiple versions of pinecone to determine which version is compatible with other requirements. This could take a while.
  Using cached pinecone-5.4.1-py3-none-any.whl.metadata (19 kB)
  Using cached pinecone-5.4.0-py3-none-any.whl.metadata (19 kB)
  Using cached langchain_pinecone-0.2.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached langchain_pinecone-0.2.1-py3-none-any.whl.metadata (1.6 kB)
INFO: pip is still looking at multiple versions of pinecone to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_pinecone-0.2.0-py3-none-any.whl.metadata (1.7 kB)
Using cached pinecone_plugin_inference-1.1.0-py3-none-any.whl (85 kB)
Using cached langchain_pinecone-0.2.0-py3-none-any.whl (11 kB)
  Attempting uninstall: pinecone-plugin-inference
    Found existing installation: pinecone-plugin-inference 3.1.0
    Uninstalling pine

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone 5.4.2 requires pinecone-plugin-inference<4.0.0,>=2.0.0, but you have pinecone-plugin-inference 1.1.0 which is incompatible.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
import os
from langchain_groq import ChatGroq

# Set API key directly
api_key = "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"

llm = ChatGroq(model_name="mixtral-8x7b-32768", temperature=0.4, max_tokens=500, api_key=api_key)


In [39]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question as thoroughly as possible. Provide a detailed "
    "and well-structured response. Avoid giving vague or overly "
    "short answers. If you don't know the answer, say that you "
    "don't know."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [40]:
# Create the question-answering chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Assume `retriever` is already defined elsewhere
rag_chain = create_retrieval_chain(retriever, question_answer_chain)  

In [46]:
# Invoke the RAG pipeline
response = rag_chain.invoke({"input": "what are the auto antibodies found in SLE?"})
print(response["answer"])

Systemic Lupus Erythematosus (SLE) is a systemic autoimmune disease, which means that it can affect multiple organs and tissues in the body. The hallmark of SLE is the presence of autoimmune manifestations that are likely to be etiologic in the organ pathology.

One of the key features of SLE is the production of a vast array of autoantibodies, which are antibodies that target the body's own cells and tissues. These autoantibodies are produced as a result of a generalized hyperreactivity of the humoral immune system, which is the part of the immune system that produces antibodies.

In addition to generalized B cell hyperresponsiveness and polyclonal hypergammaglobulinemia, which is an increase in the level of certain types of antibodies in the blood, people with SLE also often have increased titers of antibodies to commonly encountered viral antigens.

There are several specific autoantibodies that are commonly found in people with SLE. These include antibodies to double-stranded DNA (